In [1]:
import os
import re
import csv
import sys
import math
import heapq
import shutil
import queue as Q
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba
import nltk
from model import TrieNode

In [ ]:
folder = os.path.exists('blocks')
if not folder:
    os.makedirs('blocks')

In [19]:
ori_path = '动漫_简化'
files= os.listdir(ori_path)

In [41]:
def dict_union(d1, d2):
    keys = d1.keys() | d2.keys()
    temp = {}
    for key in keys:
        temp[key] = sum([d.get(key,0) for d in (d1, d2)])
    return temp

In [147]:
def construct1(intToJuji, intToFanju):
    i = 0
    j = -1000
    k = 0
    terms_id = dict()
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) == 0:
            j+=1000
            i=0
            intToFanju[j] = root
        else:
            i+=1
            key = j+i
            intToJuji[key] = root
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                texts_s,texts_l = arrayTodict(lines)
                counts = cut3(texts_s)
            for word in counts:
                terms_id[word] = terms_id.get(word, 0) + counts[word]
    return terms_id

In [39]:
def construct(intToJuji, intToFanju):
    i = 0
    j = -1000
    k = 0
    terms_id = dict()
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) == 0:
            if j>0:
                build_c(j, terms_id)
                terms_id = dict()
            j+=1000
            i=0
            intToFanju[j] = root
        else:
            i+=1
            key = j+i
            intToJuji[key] = root
            counts = dict()
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                texts_l = arrayTodict(lines)
                counts = dict_union(counts, cut1(texts_l))
            for word in counts:
                if word in terms_id:
                    terms_id[word].append((key,counts[word]))
                else: 
                    terms_id[word] = [(key,counts[word])]
    build_c(j, terms_id)
    terms_id = dict()

In [116]:
intToJuji = dict()
intToFanju= dict()

In [117]:
intToFanju

{}

In [118]:
small_words = construct(intToJuji, intToFanju)

In [149]:
small_words = sorted(small_words.items(),key = lambda item: -item[1])

In [156]:
len(new_words)

51619

In [153]:
new_words = []
for word in small_words:
    if word[1]!=1:
        new_words.append(word)

In [154]:
sys.getsizeof(small_words)

1678648

In [142]:
s = 'sjkk啥呀'

In [144]:
for i in superfluous:
    s = s.rstrip(i)

In [145]:
s

'sjkk'

In [2]:
def shorten(s):
    superfluous = ' 啊吧哇呀嘛吗耶？！?!.。诶了阿哈呗啥'
    for i in superfluous:
        s = s.rstrip(i)
    ss= ''.join(reversed(s))
    k = -1
    j = 0
    array = [-1]
    plen = len(ss)
    zeros = 0
    nonze = 0
    while j < plen:
        if k == -1 or ss[j] == ss[k]:
            k += 1
            j += 1
            if k == 0:
                if j != zeros + 1:
                    loop = (int)(nonze/zeros) -1
                    return cut_string(s, plen) if loop<1 else cut_string(s, plen - zeros*loop)
                zeros += 1
            else:
                nonze += 1
            array.append(k)
        else:
            k = array[k]
    return cut_string(s, plen) if zeros==plen else (cut_string(s, zeros) if zeros>1 else cut_string(s, zeros+1)) 
def cut_string(s, lens):
    return s[0: lens]

In [3]:
def arrayTodict(lines):
    word_dict_s={}
    word_dict_l={}
    for line in lines:
        line=shorten(line.strip('\n').replace(' ', ''))
        if len(line)<2:
            continue
        if len(line)<5:
            if word_dict_s.get(line):
                word_dict_s[line] += 1
            else:
                word_dict_s[line] = 1
            continue
        if word_dict_l.get(line):
            word_dict_l[line] += 1
        else:
            word_dict_l[line] = 1
    return word_dict_s, word_dict_l

In [4]:
def arrayTodict(lines):
    word_dict_l={}
    for line in lines:
        line=shorten(line.strip('\n').replace(' ', ''))
        if len(line)<2:
            continue
        if word_dict_l.get(line):
            word_dict_l[line] += 1
        else:
            word_dict_l[line] = 1
    return word_dict_l

In [18]:
def cut1(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        words = jieba.lcut(line)
        if len(words) > 5:
            continue
        for word in words:
            if len(word) == 1:
                continue
            else:
                counts[word] = counts.get(word, 0) + num
    return counts

In [6]:
def cut3(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        counts[line] = counts.get(line, 0) + num
    return counts

In [7]:
def cut2(word_dict):
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        if len(line)<5:
            counts[line] = counts.get(line, 0) + num
            continue
        words = jieba.lcut(line)
        for word in words:
            if len(word) == 1:
                continue
            else:
                counts[word] = counts.get(word, 0) + num
    return counts

In [115]:
def build_c(i, terms_id):
    list1 = []
    path = "chunks/chunk_"+str(i)+".txt"
    list1 = sorted(terms_id.items(), key=lambda item: item[0])
    with open(path, 'w', newline='') as f:
        for key in list1:
            list2= []
            #cnt = 0
            f.write(str(key[0]))
            for word in key[1]:
                f.write(',')
                f.write(str(word[0]))
                f.write('-')
                #cnt += word[1]
                f.write(str(word[1]))
            #f.write(',')
            #f.write(str(cnt))
            f.write('\n')
    f.close()

In [9]:
def make_dirs():
    # Create a new folder to generate inverted indexes
    folder1 = os.path.exists('chunks')
    folder2 = os.path.exists('blocks')
    if not folder1:
        os.makedirs('chunks')
    if not folder2:
        os.makedirs('blocks')

In [87]:
make_dirs()

In [94]:
shorten('2333')

'233'

In [95]:
shorten('23333333')

'233'

In [96]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}

    def inc(self, numOccur):
        self.count += numOccur

    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)
    def disp_plus(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            if child.count > 10:
                child.disp_plus(ind+1)

In [97]:
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
def updateFPtree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 判断items的第一个结点是否已作为子结点
        inTree.children[items[0]].inc(count)
    else:
        # 创建新的分支
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新相应频繁项集的链表，往后添加
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    # 递归
    if len(items) > 1:
        updateFPtree(items[1::], inTree.children[items[0]], headerTable, count)

def createFPtree(word_dict, cutted, minSup=1):
    headerTable = {}
    for trans in word_dict:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    freqItemSet = set(headerTable.keys()) # 满足最小支持度的频繁项集
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] # element: [count, node]

    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        # dataSet：[element, count]
        localD = {}
        for item in tranSet:
            if item in freqItemSet: # 过滤，只取该样本中满足最小支持度的频繁项
                localD[item] = headerTable[item][0] # element : count
        if len(localD) > 0:
            # 根据全局频数从大到小对单样本排序
            orderedItem = [v[0] for v in localD.items()]
            # 用过滤且排序后的样本更新树
            updateFPtree(orderedItem, retTree, headerTable, count)
    return retTree, headerTable

In [171]:
print ("/".join(jieba.cut("吉良吉影觉得很赞")))

吉良吉影/觉得很赞


In [10]:
new_words = []
f = open("new_words_all.txt")
lines = f.readlines()
for line in lines:
    split = line.strip().split(',')
    new_words.append((split[0],int(split[1])))
f.close()

In [11]:
new_words[0]

('233', 251716)

In [15]:
for key_word in new_words:
    if key_word[1] > 2:
        jieba.add_word(key_word[0], freq = key_word[1]*10, tag = None)

In [124]:
def build_blocks_heap(N):
    # SPIMI
    reader = txt_reader('chunks')
    heap = []
    for i in range(len(reader)):
        next_line = reader[i].readline().strip().split(',')
        heapq.heappush(heap, (next_line[0], next_line[1:], i))
    terms_id = dict()
    j = 0
    next_term = ''
    try:
        while not len(heap) == 0:
            next_index = heapq.heappop(heap)
            judge = (next_index[0] == next_term)
            next_term = next_index[0]
            next_line = reader[next_index[2]].readline()
            if next_line:
                next_line= next_line.strip().split(',')
                heapq.heappush(heap, (next_line[0], next_line[1:], next_index[2]))
            else:
                reader[next_index[2]].close()
            if (sys.getsizeof(terms_id) > 1000000 and not judge):
                j = j + 1
                build_b(j, terms_id, N)
                terms_id = dict()
            if next_term in terms_id:
                terms_id[next_term].extend(next_index[1])
            else: 
                terms_id[next_term] = next_index[1]
    finally:
        print(sys.getsizeof(terms_id))
        j=j+1
        build_b(j, terms_id, N)            

In [112]:
sys.getsizeof(terms_id)

NameError: name 'terms_id' is not defined

In [113]:
def build_b(i, terms_id, N):
    path = "blocks/block_"+ str(i) +".csv"
    with open(path, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            for key in terms_id:
                list2= []
                list2.append(key)
                idf = math.log(N /len(terms_id[key]), 10)
                list2.append(idf)
                list2.extend(terms_id[key])
                writer.writerow(list2)
        finally:
            f.close()

In [125]:
build_blocks_heap(N)

147560


In [119]:
N = len(intToJuji)

In [120]:
N

894

In [37]:
def txt_reader(path):
    fileList = os.listdir(path)
    files_reader = dict()
    i = 0
    for file in fileList:
        if(file.split('.')[-1] == "txt"):
            name = path+'/'+file
            files_reader[i] = open(name, "r")
            i = i + 1
    return files_reader

In [141]:
def solve(query, files_reader):
    words = jieba.lcut(query)
    query = []
    for word in words:
        query.append(word)
    print(query)
    format_query = dict()
    D = len(query)
    for word in query:
        if word in format_query:
            format_query[word] += 1
        else:
            format_query[word] = 1
    scores = dict()
    len1 = dict()
    len2 = 0
    for i in range(len(files_reader)):
        with open(files_reader[i+1]) as f:
            lines = csv.reader(f)
            k = 0
            for line in lines:
                k+=1
                if line[0] in format_query:
                    idf1 = float(line[1])
                    w2 = (1+math.log(int(format_query[line[0]]), 10)) * idf1
                    for term in line[2:]:
                        tmp = term.split('-')
                        w1 = (1+math.log(int(tmp[1]), 10)) * idf1
                        if tmp[0] in scores:
                            scores[tmp[0]] += w1 * w2
                        else: 
                            scores[tmp[0]] =  w1 * w2
                            len1[tmp[0]] = 1
                    len2 += w2 * w2
        f.close()
    for i in range(len(files_reader)):
        with open(files_reader[i+1]) as f:
            lines = csv.reader(f)
            for line in lines:
                for word in line[2:]:
                    tmp = word.split('-')
                    if tmp[0] in len1:
                        w1 = (1+math.log(int(tmp[1]), 10)) * float(line[1])
                        len1[tmp[0]] += w1 * w1
        f.close()       
    for score in scores:
        scores[score] =scores[score] / math.sqrt(len1[score] * len2)
    list1= sorted(scores.items(), key=lambda x:x[1], reverse=True)  
    for answer in find_answer(list1, filename_index):
        print(answer)

In [142]:
def find_answer(scores, filename_index):
    answer = []
    i = 0
    for score in scores:
        if i == 10:
            return answer
        i += 1
        answer.append(filename_index[int(score[0])])
    return answer

In [133]:
path = 'blocks'
fileList = os.listdir(path)
files_reader = dict()
for file in fileList:
    if(file.split('.')[-1] == "csv"):
        name = path+'/'+file
        com = re.compile(r"_(\d+).csv", re.M)
        find = com.findall(name)
        i = int(find[0])
        files_reader[i] = name

In [143]:
solve('寄刀片', files_reader)

['寄刀片']
动漫_简化\JOJO的奇妙冒险-黄金之风\38_镇魂歌沉静地演奏着 其二
动漫_简化\辉夜大小姐想让我告白天才们的恋爱头脑战\12_听不见烟花的声音后篇辉夜大小姐不想逃避
动漫_简化\JOJO的奇妙冒险黄金之风\38_镇魂歌沉静地演奏着其二
动漫_简化\JOJO的奇妙冒险-黄金之风\36_其名为迪亚波罗
动漫_简化\JOJO的奇妙冒险-星尘斗士-埃及篇\22_迪奥的世界 其二
动漫_简化\JOJO的奇妙冒险\20_西撒的孤独青春
动漫_简化\JOJO的奇妙冒险黄金之风\31_总集篇JOJO的奇妙冒险黄金之风命运
动漫_简化\JOJO的奇妙冒险-黄金之风\31_总集篇：JOJO的奇妙冒险 黄金之风 命运
动漫_简化\辉夜大小姐想让我告白天才们的恋爱头脑战\11_早坂爱想泡澡藤原千花超想吃白银御行想见面听不见烟花的声音前篇
动漫_简化\JOJO的奇妙冒险-黄金之风\30_在那看似即将坠落的天空下


In [52]:
filename_index = intToJuji

In [65]:
len(intToJuji)

894

In [137]:
intToFanju

{0: '动漫_简化',
 1000: '动漫_简化\\AIR',
 2000: '动漫_简化\\CLANNAD',
 3000: '动漫_简化\\JOJO的奇妙冒险',
 4000: '动漫_简化\\JOJO的奇妙冒险-不灭钻石',
 5000: '动漫_简化\\JOJO的奇妙冒险-星尘斗士',
 6000: '动漫_简化\\JOJO的奇妙冒险-星尘斗士-埃及篇',
 7000: '动漫_简化\\JOJO的奇妙冒险-黄金之风',
 8000: '动漫_简化\\JOJO的奇妙冒险黄金之风',
 9000: '动漫_简化\\月色真美',
 10000: '动漫_简化\\来自新世界',
 11000: '动漫_简化\\游☆戏☆王5Ds第四季',
 12000: '动漫_简化\\游☆戏☆王GX',
 13000: '动漫_简化\\游戏王怪兽之决斗',
 14000: '动漫_简化\\辉夜大小姐想让我告白天才们的恋爱头脑战',
 15000: '动漫_简化\\高机动幻想GPO第二季',
 16000: '动漫_简化\\高机动幻想～新结行军歌～',
 17000: '动漫_简化\\高达创形者',
 18000: '动漫_简化\\高达创形者再起',
 19000: '动漫_简化\\高达创战者',
 20000: '动漫_简化\\鬼灭之刃',
 21000: '动漫_简化\\魔法少女小圆',
 22000: '动漫_简化\\魔法纪录魔法少女小圆外传',
 23000: '动漫_简化\\魔法记录-魔法少女小圆外传'}